In [32]:
:set -XOverloadedStrings

import qualified Data.ByteString as B

import Control.Monad.Trans.Reader
import Control.Monad.IO.Class (liftIO)
import Control.Monad (mapM_, unless)
import System.Directory
import System.FilePath

import Duffer.Loose

duffer = flip runReaderT "output/.git"
git    = flip runReaderT ".git"

In [31]:
Just o <- git$ readObject "b1bbf2b4b38a53c8b986d30b0e8f60ff8c01505f"

initRepo = do
    path <- ask
    objectsPresent <- liftIO $ doesDirectoryExist $ path </> "objects"
    liftIO $ unless objectsPresent $ do
        mapM_ (createDirectoryIfMissing True)
            [ path </> "branches"
            , path </> "hooks"
            , path </> "info"
            , path </> "objects" </> "info"
            , path </> "objects" </> "pack"
            , path </> "refs"    </> "heads"
            , path </> "refs"    </> "tags"
            ]
        mapM_ (uncurry writeFile)
            [ (path </> "HEAD",   "ref: refs/heads/master\n")
            , (path </> "config", "")
            , (path </> "description", "")
            ]

destroyRepo = do
    path <- ask
    liftIO $ removeDirectoryRecursive path


_ <- duffer initRepo